Interface for the Crossref.org API

To get out the Journals DOI

In [2]:
version = "1.0.0"
datadir = "data/"
archivedir = "data/Archive"

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import datetime
import pandas as pd
from time import sleep
import requests
import random


In [3]:


journals = pd.read_csv('doaj_journals.csv', sep="|")
journals.drop_duplicates(inplace=True)

In [4]:
# search  for IOP
iop = journals[journals['publisher'] == "IOP Publishing"]
iop = iop.to_dict('records')

In [5]:
print(iop)

[{'title': 'Environmental Research Letters', 'publisher': 'IOP Publishing', 'eissn': '1748-9326', 'pissn': nan, 'doajid': 'eddad91945dd48c29143fa50ee6e3846', 'categories': 'Environmental technology. Sanitary engineering#Environmental sciences#Science#Physics', 'language': 'EN'}, {'title': 'Journal of Physics Communications', 'publisher': 'IOP Publishing', 'eissn': '2399-6528', 'pissn': nan, 'doajid': 'f2936121500f4a849f49571f17312206', 'categories': 'Science#Physics', 'language': 'EN'}, {'title': 'Materials Research Express', 'publisher': 'IOP Publishing', 'eissn': '2053-1591', 'pissn': nan, 'doajid': '45bf0e93caae4c039d0938713d107163', 'categories': 'Materials of engineering and construction. Mechanics of materials#Chemical technology', 'language': 'EN'}, {'title': 'New Journal of Physics', 'publisher': 'IOP Publishing', 'eissn': '1367-2630', 'pissn': nan, 'doajid': 'c63ca2415e0945fa9bdafe2d82b4ef84', 'categories': 'Science#Physics', 'language': 'EN'}, {'title': 'Environmental Researc

In [ ]:

#searchterm = "https://api.crossref.org/journals/1367-2630/works?filter=type:journal-article"
searchterm = "https://api.crossref.org/journals/1367-2630/works?filter=type:journal-article"
mailto = "&mailto=sajozsattila@mur2.co.uk"
print(searchterm)
#x = requests.get(searchterm+"&cursor=*")
#x = requests.get(searchterm+"?cursor=*")
x = requests.get(searchterm+"&rows=1000")
x = x.json()


In [ ]:

def processresult(x, journal, df):
    articles = []
    newitem = True
    if isinstance(x['message'], dict) and 'items' in  x['message'].keys() and isinstance(x['message']['items'], list):
        for d in x['message']['items']:
            article = { 'title': d['title'][0], 'doi': d['DOI']}
    
            # writer
            writer = []
            if 'author' in d.keys():            
                for w in d['author']:
                    try:
                        writer.append( w['given']+","+w['family'] )
                    except:
                        try:
                            writer.append( w['name'] )
                        except Exception as e:
                            print("error in writer", str(w))
                            continue
                if 'ORCID' in w.keys():
                    writer[-1] = writer[-1] + "---" + w['ORCID'].replace('http://orcid.org/', '')
            article['writer'] = "#".join(writer)
    
            # date
            article['publishdate'] = "{:4d}-{:02d}-{:02d}".format( 
                d['indexed']['date-parts'][0][0],
                d['indexed']['date-parts'][0][1],
                d['indexed']['date-parts'][0][2]
                 )
    
            # url 
            try:
                if 'URL' in d.keys():
                     article['url'] = d['URL']
                elif 'link' in d.keys() and isinstance(d['link'], list):
                    article['url'] = d['link'][0]['URL']
                else:
                    print("Errorl in link1", d['link'])

            except Exception as e:
                print(e)
                print("Errorl in link", d)
    
            # publisher
            article['journal_title'] = journal['title']
            article['journal_eissn'] = journal['eissn']
            article['journal_pissn'] = journal['pissn']
            article['category'] = journal['categories']
            article['keyword'] = ""
            article['abstract'] = ""
    
            articles.append(article)
            tmpdf = pd.DataFrame(articles)
        print(len(tmpdf))
        if len(tmpdf) < 10:
            newitem = False
        df = pd.concat([df, tmpdf])
        df.reset_index(drop=True, inplace=True)
        return df, newitem
    
    else:
        return None, False

        
    

journal = [ i for i in iop if i['title'] == 'New Journal of Physics' ][0]
filename = 'predata/journal_'+journal['title'].replace(" ", "_").replace(":", "_")+'_'+version+'.pandas'

df = pd.DataFrame({})
print(len(df))
df, _ = processresult(x, journal, df)
df.to_pickle(filename)
print(len(df))
# go to next
pagecounter = 1
pagelength = 1000
while True:
    newpage = None
    try:
        # nextpage = x['message']['next-cursor']
        # print(searchterm+"&cursor="+nextpage)
        # x = requests.get(searchterm+"&cursor="+nextpage)
        # x = requests.get(searchterm+"?cursor="+nextpage)
        x = requests.get(searchterm+"&rows="+str(pagelength)+"&offset="+str(pagecounter*pagelength))
        pagecounter = pagecounter+1
        x = x.json()        
    except Exception as e:
        print(e)
        break
    df, newround = processresult(x, journal, df)
    if not newround:
        break
    
    # save 
    df.to_pickle(filename)
    print(len(df))
    
    # random sleep
    sleep(random.uniform(15, 45))

In [8]:
filename = 'predata/journal_New_Journal_of_Physics_1.0.0.pandas'
df = pd.read_pickle(filename)
print(df.tail())


# iterate over the articles and try to get the abstract
ji = 0
for _ in range(len(df)):
    if ji > len(df):
        break
    url = df.iloc[ji]['url']
    response = urlopen(url)
    content = response.read()
    print(content)
    page_soup = BeautifulSoup(content, features="lxml")
    
    # abstract
    abstract = ""
    for abstdiv in page_soup.find_all("div", {'class':['wd-jnl-art-abstract']}):
        for ap in abstdiv.find_all("p"):
            abstract = abstract + (" ".join(ap.get_text().splitlines()
                                           ).replace("\xa0", " ") ).replace("Abstract", '')
    if len(abstract) == 0:
        print("No abstract", url)
        sleep(random.uniform(150, 450))
        continue

    keywords = ""    
    for m in page_soup.find_all("meta"):
        if m.has_attr("name"):
            if m['name'] == "dc.Subject":
                keywords = m['content'].replace(";", "#")
    
    df.iloc[ji]['abstract'] = abstract
    df.iloc[ji]['keyword'] = keywords
    df.to_pickle(filename)
    
    ji = ji+1
    
    sleep(random.uniform(150, 450))

                                                   title  \
10995  How to estimate the differential acceleration ...   
10996  Test for bacterial resistance build-up against...   
10997  How to reveal metastable skyrmionic spin struc...   
10998  Charge dynamics of the antiferromagnetically o...   
10999  Spin-polarized current induced by a local exch...   

                                  doi  \
10995  10.1088/1367-2630/11/11/113010   
10996   10.1088/1367-2630/14/7/073037   
10997   10.1088/1367-2630/18/5/055015   
10998  10.1088/1367-2630/18/10/103004   
10999   10.1088/1367-2630/14/8/083039   

                                                  writer publishdate  \
10995  G,Varoquaux#R A,Nyman#R,Geiger#P,Cheinet#A,Lan...  2020-11-13   
10996  J L,Zimmermann#T,Shimizu#H-U,Schmidt#Y-F,Li#G ...  2020-11-13   
10997               B,Dupé#C N,Kruse#T,Dornheim#S,Heinze  2020-11-13   
10998  Xing-Jie,Han#Yu,Liu#Zhi-Yuan,Liu#Xin,Li#Jing,C...  2020-11-13   
10999  Xing-Tao,An#Yan-Yang,Zhan

KeyboardInterrupt: 